<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_15/S15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Install packages

In [11]:
#!git clone "https://github.com/jyanivaddi/ERA_V1.git"
#!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull
#!cd ../

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts" "torchtext" "datasets" "tokenizers"
!pip install --quiet "prettytable"
!pip install --quiet "torchsummary"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"
!pip install --quiet "gradio"

fatal: destination path 'dl_hub' already exists and is not an empty directory.
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 9 (delta 6), reused 9 (delta 6), pack-reused 0
Unpacking objects: 100% (9/9), 4.27 KiB | 1.42 MiB/s, done.
From https://github.com/jyanivaddi/dl_hub
   fe038ff..2af9fac  main       -> origin/main
Updating fe038ff..2af9fac
Fast-forward
 Transformer/PL_data_module.py |  68 +++++++++++++++---------
 Transformer/PL_main.py        |  44 ++++++++++++++--
 Transformer/PL_model.py       |  46 ++++++++++-------
 Transformer/config.py         |   9 ++--
 Transformer/dataset.py        | 117 +++++++++++++++++++++++++++++++++++++++++-
 Transformer/model.py          |  39 ++++++++++----
 6 files changed, 262 insertions(+), 61 deletions(-)
Already up to date.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A 

In [12]:
import sys
sys.path.append("ERA_V1/session_15")
sys.path.append("dl_hub")

In [13]:
!git -C dl_hub pull


Already up to date.


In [14]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from dl_hub.Transformer.config import get_config
from dl_hub.Transformer.PL_data_module import OpusDataModule
from dl_hub.Transformer.PL_model import LitTransformer
from dl_hub.Transformer.PL_main import train_transformer



In [2]:
#from config import get_config
#from PL_data_module import OpusDataModule
#from PL_model import LitTransformer
#from PL_main import train_transformer


**Finetune config params**

In [15]:
cfg = get_config()
cfg["parameter_sharing"] = True
cfg["d_ff"] = 512
cfg["batch_size"] = 48
cfg["d_model"] = 512
cfg["preload"] = None
cfg["num_epochs"] = 30
cfg["ds_mode"] = 'disk'
cfg["lang_tgt"] = 'fr'
cfg["ds_path"] = "/home/e183534/ERA_Datasets/OpusBooks_En_Fr"
cfg["save_ds_to_disk"] = False
cfg["experiment_name"] = "tmodel_dynamic_pad_150_tokens_bs_32_amp_ps_dff_512"



**Mixed precision training**

In [16]:
torch.cuda.amp.autocast(enabled=True)

**Define data module**

In [17]:
opus_dm = OpusDataModule(cfg)
opus_dm.prepare_data()
opus_dm.setup()
train_data_loader = opus_dm.train_dataloader()
#pp = next(iter(train_data_loader))

**Define PL Model**

In [7]:
tokenizer_src = opus_dm.tokenizer_src
tokenizer_tgt = opus_dm.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = LitTransformer(cfg, loss_criterion, tokenizer_src, tokenizer_tgt, num_validation_examples=2, epochs = cfg['num_epochs'])
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)

scheduler = OneCycleLR(
    optimizer,
    max_lr=1E-3,
    steps_per_epoch=len(train_data_loader),
    epochs=cfg['num_epochs'],
    pct_start=5/cfg['num_epochs'],
    div_factor=10,
    three_phase=True,
    final_div_factor=10,
    anneal_strategy='linear'
)
transformer_model.set_scheduler_dict(scheduler, freq = 'step')

Total Parameters: 58699090


In [8]:
train_transformer(transformer_model, opus_dm, cfg, epochs=cfg['num_epochs'])

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 58.7 M
----------------------------------------------------
58.7 M    Trainable params
0         Non-trainable params
58.7 M    Total params
117.398   Total estimated model params size (MB)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: « Eh bien ! c’était un homme outrageusement ivre.
    TARGET: "He was an uncommon drunk sort o' man," he said.
 PREDICTED: " " What , and the .
*****************************************

*****************************************
    SOURCE: The owl often assumes this oblique attitude.
    TARGET: Le hibou a souvent cette attitude oblique.
 PREDICTED: Le , cette se .
*****************************************


/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:61: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


...

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: "But you _will_ know it, when I tell you what happened the very next day."
    TARGET: – Vous ne penserez plus de meme lorsque vous saurez la suite.
 PREDICTED: -- Mais vous le savez , quand je vous dirai ce qui est le lendemain même .
*****************************************

*****************************************
    SOURCE: His figure was enveloped in a riding cloak, fur collared and steel clasped; its details were not apparent, but I traced the general points of middle height and considerable breadth of chest.
    TARGET: Il était enveloppé d'une redingote à collet de fourrure et à boutons d'acier; je ne pus pas remarquer les détails, mais je vis l'ensemble.
 PREDICTED: Sa figure était gonflée au manteau , d ' acier et joignit en un haut éclat ; ses détails n ' étaient pas nombreux , mais je pus les côtés de la taille , et considérable de poitrine .
*****************************************

Epoch 29: train_loss=1.7107


`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

*****************************************
    SOURCE: 'Well, gentlemen, I shall be the third parish priest, eighty years of age, to be deprived of his living in this district.
    TARGET: – Eh bien, messieurs ! je serai le troisième curé, de quatre-vingts ans d’âge, que l’on destituera dans ce voisinage.
 PREDICTED: – Eh bien , messieurs , j ’ aurai le troisième curé , quatre - vingt ans d ’ âge , se dans ce pays .
*****************************************

*****************************************
    SOURCE: Tipsy from that potent juice, our bird of paradise had been reduced to helplessness.
    TARGET: Le paradisier, enivré par le suc capiteux, était réduit à l'impuissance.
 PREDICTED: L ' oiseau du paradis était lourd et profond .
*****************************************
